In [17]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore as pos
import bte

First, we load the tree and the label data into memory.

In [34]:
t = bte.MATree("seabra_zika.autolin.pb")
t

Finished 'from_pb' in 0.0281 seconds


MATree object with 767 leaves.

In [3]:
df = pd.read_csv("seabra_zika.labels.txt",sep='\t',names=['lineage','strain'])[['strain','lineage']]
df

,strain,lineage
0,KY288905_Uganda_1962-11,L
1,OK054351_India_2021-07-28,L
2,KX377336_Malaysia_1966-07,L
3,MN025403_Guinea_2018-08,L
4,KX601166_Senegal_1984-11-17,L
...,...,...
3550,OK054466_Nicaragua_2016-07-13,auto.L.1.1
3551,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1
3552,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1.1
3553,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1.1.2


For visualization convenience downstream, we reshape the label table by splitting the lineages into their various levels; the resulting CSV can be included in the creation of a protocol buffer or dragged-and-dropped onto Auspice to allow the user to color by any one of the levels of annotation.

In [4]:
counts = []
for l in df.lineage:
    elements = l.split(".")
    if elements[0] == 'auto':
        elements = elements[1:]
    counts.append(len(elements))
df['counts'] = counts
df.counts

0       1
1       1
2       1
3       1
4       1
       ..
3550    3
3551    4
3552    5
3553    6
3554    7
Name: counts, Length: 3555, dtype: int64

Only retain the highest level (max counts) entry for each strain.

In [67]:
df = df.dropna().sort_values("counts",ascending=False).drop_duplicates("strain")

In [68]:
linslots = [[] for i in range(0,df.counts.max())]
for l in df.lineage:
    elements = l.split(".")
    if elements[0] == 'auto':
        elements = elements[1:]
    for i in range(0,df.counts.max()):
        current = ".".join(elements[:i+1])
        if current == "L":
            current = "not assigned"
        linslots[i].append(current)
for i in range(1,len(linslots)):
    index = "GRI Lineage Level " + str(i)
    df[index] = linslots[i]
df

,strain,lineage,counts,GRI Lineage Level 1,GRI Lineage Level 2,GRI Lineage Level 3,GRI Lineage Level 4,GRI Lineage Level 5,GRI Lineage Level 6,SeabraLineage
3554,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1.1.2.1,7,L.1,L.1.1,L.1.1.1,L.1.1.1.1,L.1.1.1.1.2,L.1.1.1.1.2.1,ZB.2.1
2420,MK028861_Panama_2015,auto.L.1.1.5.1.2.1,7,L.1,L.1.1,L.1.1.5,L.1.1.5.1,L.1.1.5.1.2,L.1.1.5.1.2.1,ZB.2.0
2835,OK054450_Nicaragua_2016-08-24,auto.L.1.1.1.4.1.1,7,L.1,L.1.1,L.1.1.1,L.1.1.1.4,L.1.1.1.4.1,L.1.1.1.4.1.1,ZB.2.1
2828,OK054436_Nicaragua_2016-07-14,auto.L.1.1.1.4.1.1,7,L.1,L.1.1,L.1.1.1,L.1.1.1.4,L.1.1.1.4.1,L.1.1.1.4.1.1,ZB.2.1
1910,OK054392_Nicaragua_2016-07-27,auto.L.1.1.1.16.1.1,7,L.1,L.1.1,L.1.1.1,L.1.1.1.16,L.1.1.1.16.1,L.1.1.1.16.1.1,ZB.2.1
...,...,...,...,...,...,...,...,...,...,...
22,KY328290_Bangladesh_2016-11-03,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZB.1.1
23,MT377498_Thailand_2016-04-26,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZB.1.1
17,KU179098_Indonesia_2014-12-30,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZB.1.1
4,KX601166_Senegal_1984-11-17,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZA


In [69]:
df.to_csv("seabra_zika.labels.split.tsv",sep='\t',index=False)

We retrieve Seabra's nomenclature from their [supplementary data](https://academic.oup.com/ve/article/8/1/veac029/6555351#351081937).

In [12]:
sdf = pd.read_excel("Supplementary_Table_S3_NEW.xlsx").set_index("id_country_date")
sdf

,order,id,seq_length,host,host_species,ATCG%,CG%,CountryCleaned,SamplingDate_formatted,Year,...,Hb_759_level2,Hb_752_level1,Hb_752_level2,groups_hb759,groups_hb752,NAMING_PROPOSAL,ids_iTOL,colour_regions_itol,colour_host_itol,colour_naming_itol
id_country_date,,,,,,,,,,,,,,,,,,,,,
OK571913_Haiti_2014-11-25,5,OK571913,10807,Human,Homo sapiens,100.000000,51.095530,Haiti,2014-11-25,2014,...,1.0,1.0,1.0,ZB.2.0,ZB.2.0,ZB.2.0,OK571913_Haiti_2014-11-25,#33a02c,#d9d9d9,#1f78b4
MW123921_Brazil_2016-04-26,6,MW123921,10639,Human,Homo sapiens,100.000000,51.124744,Brazil,2016-04-26,2016,...,2.0,1.0,1.0,ZB.2.0,ZB.2.0,ZB.2.0,MW123921_Brazil_2016-04-26,#1f78b4,#d9d9d9,#1f78b4
MW123922_Brazil_2016-05-10,7,MW123922,10639,Human,Homo sapiens,97.546012,51.093142,Brazil,2016-05-10,2016,...,2.0,1.0,1.0,ZB.2.0,ZB.2.0,ZB.2.0,MW123922_Brazil_2016-05-10,#1f78b4,#d9d9d9,#1f78b4
MW123923_Brazil_2016-06-16,8,MW123923,10651,Human,Homo sapiens,99.045671,50.968440,Brazil,2016-06-16,2016,...,2.0,1.0,1.0,ZB.2.0,ZB.2.0,ZB.2.0,MW123923_Brazil_2016-06-16,#1f78b4,#d9d9d9,#1f78b4
MW123924_Brazil_2016-05-07,9,MW123924,10648,Human,Homo sapiens,96.737754,51.016710,Brazil,2016-05-07,2016,...,2.0,1.0,1.0,ZB.2.0,ZB.2.0,ZB.2.0,MW123924_Brazil_2016-05-07,#1f78b4,#d9d9d9,#1f78b4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KF268950_CentralAfricanRepublic_1976,2029,KF268950,10755,Mosquito,Aedes africanus,99.951310,50.448168,CentralAfricanRepublic,1976,1976,...,11.0,NaN,NaN,ZA,NaN,ZA,KF268950_CentralAfricanRepublic_1976,#fdbf6f,#252525,#c1420c
AY632535_Uganda_1947,2175,AY632535,10794,Monkey,sentinel monkey,99.883143,50.784830,Uganda,1947,1947,...,10.0,NaN,NaN,ZA,NaN,ZA,AY632535_Uganda_1947,#c1420c,#806a61,#c1420c
KY241700_Singapore_2016-08-27,1570,KY241700,10738,Human,Homo sapiens,99.522836,51.144814,Singapore,2016-08-27,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,KY241700_Singapore_2016-08-27,#6a3d9a,#d9d9d9,NaN


Their strains are in the "NAMING_PROPOSAL" column. We transfer these names into the label table.

In [18]:
def get_np(strain):
    try:
        return sdf.loc[strain].NAMING_PROPOSAL
    except KeyError:
        return np.nan
df['SeabraLineage'] = df.strain.apply(get_np)
df.dropna(inplace=True)
df

,strain,lineage,counts,GRI Lineage Level 1,GRI Lineage Level 2,GRI Lineage Level 3,GRI Lineage Level 4,GRI Lineage Level 5,GRI Lineage Level 6,SeabraLineage
0,KY288905_Uganda_1962-11,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZA
1,OK054351_India_2021-07-28,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZB.1.0
2,KX377336_Malaysia_1966-07,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZB.1.0
3,MN025403_Guinea_2018-08,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZA
4,KX601166_Senegal_1984-11-17,L,1,not assigned,not assigned,not assigned,not assigned,not assigned,not assigned,ZA
...,...,...,...,...,...,...,...,...,...,...
3550,OK054466_Nicaragua_2016-07-13,auto.L.1.1,3,L.1,L.1.1,L.1.1,L.1.1,L.1.1,L.1.1,ZB.2.1
3551,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1,4,L.1,L.1.1,L.1.1.1,L.1.1.1,L.1.1.1,L.1.1.1,ZB.2.1
3552,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1.1,5,L.1,L.1.1,L.1.1.1,L.1.1.1.1,L.1.1.1.1,L.1.1.1.1,ZB.2.1
3553,OK054466_Nicaragua_2016-07-13,auto.L.1.1.1.1.2,6,L.1,L.1.1,L.1.1.1,L.1.1.1.1,L.1.1.1.1.2,L.1.1.1.1.2,ZB.2.1


We can now compute the ARI between different levels of annotation and the Seabra nomenclature. Level 3 is generally the most concordant, representing lineages of about the same size.

In [70]:
for i in range(1, 7):
    ln = "GRI Lineage Level " + str(i)
    print(ln, ari(df.SeabraLineage, df[ln]))

GRI Lineage Level 1 0.06295406079320065
GRI Lineage Level 2 0.22168505431544977
GRI Lineage Level 3 0.467394449314314
GRI Lineage Level 4 0.08716677740542479
GRI Lineage Level 5 0.06346404086422941
GRI Lineage Level 6 0.061138993113729063


We now test whether this value is significantly higher than expected through a simple permutation test. This is the same style of permutation test used for CHIKV and VEEV in this manuscript.

In [38]:
t = t.subtree(df.strain)

Extracting subtree of 3526 samples.


No input arguments passed; creating default tree.


Completed in 547 msec 

Finished 'subtree' in 0.553 seconds


In [39]:
t

MATree object with 759 leaves.

In [40]:
node_ids = [n.id for n in t.depth_first_expansion() if not n.is_leaf()]
len(node_ids)

757

In [51]:
def permute_ari():
    node_choices = np.random.choice(node_ids,size=df.lineage.nunique())
    fake_lineage_labels = []
    for strain in df.strain:
        tagged = False
        for anc in t.rsearch(strain):
            if anc.id in node_choices:
                fake_lineage_labels.append(anc.id)
                tagged = True
                break
        if not tagged:
            fake_lineage_labels.append('not assigned')
    return ari(df.SeabraLineage, fake_lineage_labels)
permute_ari()

0.09880911120505816

In [63]:
perm_dist = [permute_ari() for i in range(10000)]

In [64]:
np.mean(perm_dist)

0.08499804441540525

In [66]:
pos(perm_dist, ari(df.SeabraLineage, df['GRI Lineage Level 3']))

100.0

We find that the ARI is much higher than expected at random.